# Letter to the editor: a note on “A New Extended Rayleigh Distribution”

authors: [Luiz R. Nakamura](luiz.rn@gmail.com), [Thiago G. Ramires](thiagogentil@gmail.com) and [Ana J. Righetto](ajrighetto@gmail.com)

Here we present some review and results of the paper: 
A. A. Al-Babtain, A new extended Rayleigh distribution, Journal of King Saud University – Science, https://doi.org/10.1016/j.jksus.2020.04.015


All results obtained in this review was obtained using the software R, where the codes are shown below, step by step.  

# Getting started

* First, we load the data (gauge lengths of 10 mm) obtained from D. Kundu and MZ Raqab (2009).
* Then, we construct the probability density function (pdf) and cumulative density fuctuion (cdf) of the Rayleigh (R) and type I half-logistic Rayleigh (TIHLR) models. 



In [2]:
#data set
x <- c(1.901, 2.132, 2.203, 2.228, 2.257, 2.350, 2.361, 2.396, 2.397, 2.445, 2.454, 2.474, 2.518, 2.522, 
       2.525, 2.532, 2.575, 2.614, 2.616, 2.618, 2.624, 2.659, 2.675, 2.738, 2.740, 2.856, 2.917, 2.928, 
       2.937, 2.937, 2.977, 2.996, 3.030, 3.125, 3.139, 3.145, 3.220, 3.223, 3.235, 3.243, 3.264, 3.272, 
       3.294, 3.332, 3.346, 3.377, 3.408, 3.435, 3.493, 3.501, 3.537, 3.554, 3.562, 3.628, 3.852, 3.871, 
       3.886, 3.971, 4.024, 4.027, 4.225, 4.395, 5.020)

#Rayleigh cdf and pdf
cdf0=function(par,x){
  alpha=par[1]; 
  1-exp(-alpha*x^2)}

pdf0=function(par,x){
  alpha=par[1]; 
  2*alpha*x*exp(-alpha*x^2)}


#TIHLR cdf and pdf
cdf1=function(par,x){
  alpha=par[1];  lambda=par[2]
  (1-exp(-alpha*lambda*x^2))/(1+exp(-alpha*lambda*x^2))}

pdf1=function(par,x){
  alpha=par[1];  lambda=par[2]
  (4*lambda*alpha*x*exp(-alpha*lambda*x^2))/((1+exp(-alpha*lambda*x^2))^2)}



Now, we refit the results presented in [Table 2](https://doi.org/10.1016/j.jksus.2020.04.015). To make it easier, we are going to use the `AdequacyModel` package. An important note here is that the estimation methods may not be the same, so we will consider the values presented in the paper in a future step.

In [ ]:
#refitting
library(AdequacyModel)

#Rayleigh
R=goodness.fit(pdf=pdf0, cdf=cdf0,
             starts = c(0.029),data=x,
             lim_inf = c(0),lim_sup = c(10),
             domain=c(0,Inf),method='SANN')

#TIHLR
TI=goodness.fit(pdf=pdf1, cdf=cdf1,
             starts = c(0.029,5.267),data=x,
             lim_inf = c(0,0),lim_sup = c(10,10),
              domain=c(0,Inf),method="SANN")


Comparing with the values presented in Table 2. We may note that the MLEs are not the same, but they are close. 

In [ ]:
results1=cbind(alpha=c(0.029,0.103,round(TI$mle[1],3),round(R$mle,3)),
               lambda=c(5.267,'-',round(TI$mle[2],3),'-'))
row.names(results1)=c('TIHLR (paper)','R (paper)','TIHLR (refitted)','R (refitted)')
results1


We now compare the goodness of measures for estimates presented on [Table 3](https://doi.org/10.1016/j.jksus.2020.04.015). Here we may note some inconsistencies.

In [ ]:
results2=cbind(AIC=c(173.4,round(TI$AIC,1),189.0,round(R$AIC,1)),
               CAIC=c(173.6,round(TI$`CAIC `,1),189.1,round(R$`CAIC `,1)),
               BIC=c(173.0,round(TI$BIC,1),188.8,round(R$BIC,1)),
               HQIC=c(175.1,round(TI$HQIC,1),189.8,round(R$HQIC,1)),
               A=c(18.49,round(TI$A,1),32.8,round(R$A,1)),
               W=c(1.2,round(TI$W,1),1.7,round(R$W,1))
               )
row.names(results2)=c('TIHLR (paper)','TIHLR (refitted)','R (paper)','R (refitted)')
results2


As mentioned earlier, we will consider the estimates presented on paper to obtain the goodness-of-fit measures (we only focus here on the TIHLR model). Once again we may note some incnsistencies. 

In [ ]:
TIHLR=goodness.fit(pdf=pdf1, cdf=cdf1,
             mle = c(0.029,5.267),data=x,
             lim_inf = c(0,0),lim_sup = c(10,10),
             domain=c(0,Inf))
TIHLR

As the authors proposed a new model, justifying that it is more flexible, we will compare the results with well-known and simple models, e.g., gama, log-normal and  inverse Gausian distributions. As we may note here, all of these models presented better fits.

In [ ]:
#Comparing the results with other simpler models

#gama distribution
pdf2=function(par,x){
  mu=par[1];sigma=par[2]
  dGA(x,exp(mu),exp(sigma))}
cdf2=function(par,x){
  mu=par[1];sigma=par[2]
  pGA(x,exp(mu),exp(sigma))}


#log-normal distribution
pdf3=function(par,x){
  mu=par[1];sigma=par[2]
  dLOGNO(x,exp(mu),exp(sigma))}
cdf3=function(par,x){
  mu=par[1];sigma=par[2]
  pLOGNO(x,exp(mu),exp(sigma))}


#inverse g distribution
pdf4=function(par,x){
  mu=par[1];sigma=par[2]
  dIG(x,exp(mu),exp(sigma))}
cdf4=function(par,x){
  mu=par[1];sigma=par[2]
  pIG(x,exp(mu),exp(sigma))}


GA=goodness.fit(pdf=pdf2, cdf=cdf2,
                   starts = c(3,0.2),data=x,
                   lim_inf = c(-10,-10),lim_sup = c(10,10),
                   domain=c(0,Inf),method="SANN")
GA

LN=goodness.fit(pdf=pdf3, cdf=cdf3,
                   starts = c(0.09,-1.62),data=x,
                   lim_inf = c(-10,-10),lim_sup = c(10,10),
                   domain=c(0,Inf))
LN

IG=goodness.fit(pdf=pdf4, cdf=cdf4,
                   starts = c(1,-3),data=x,
                   lim_inf = c(-10,-10),lim_sup = c(10,10),
                   domain=c(0,1000))
IG

Finally, we compare the adjustments graphically.

In [ ]:

hist(x, probability = T, main='', ylim=c(0,.7), col='lightgrey')
curve(dGA(x,exp(GA$mle[1]),exp(GA$mle[2])), add=T, lwd=2, col='red')
curve(pdf1(par =c(0.029,5.267),x),add=T,lwd=3)
curve(pdf0(par =c(0.103),x),add=T,lwd=3,col=3)
legend('topright',c("GA",'TIHLR','R'),lwd=3,bty='n',col=c(2,1,3))


hist(x, probability = T, main='', ylim=c(0,.7), col='lightgrey')
curve(dLNO(x,exp(LN$mle[1]),exp(LN$mle[2])), add=T, lwd=2, col='red')
curve(pdf1(par =c(0.029,5.267),x),add=T,lwd=3)
curve(pdf0(par =c(0.103),x),add=T,lwd=3,col=3)
legend('topright',c("LN",'TIHLR','R'),lwd=3,bty='n',col=c(2,1,3))


hist(x, probability = T, main='', ylim=c(0,.7), col='lightgrey')
curve(dIG(x,exp(IG$mle[1]),exp(IG$mle[2])), add=T, lwd=2, col='red')
curve(pdf1(par =c(0.029,5.267),x),add=T,lwd=3)
curve(pdf0(par =c(0.103),x),add=T,lwd=3,col=3)
legend('topright',c("IG",'TIHLR','R'),lwd=3,bty='n',col=c(2,1,3))